In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import sympy as sp


class Person:
    def __init__(self, name: str, foresight: int):
        self.name = name
        self.foresight = foresight
        self.decision_list = []

    def determine_optimal_choice(
        self, choice_array: np.ndarray, decision_number: int, number_of_decisions: int
    ):
        # Get the row index of the highest sum of values in the choice array up to the person's "foresight" index
        highest_value_index = np.argmax(
            np.sum(choice_array[:, : self.foresight], axis=1)
        )

        # Extract this row from the choice array
        optimal_choice = choice_array[highest_value_index, :]

        # Pad initial values with nan
        optimal_choice_padded = np.pad(
            optimal_choice,
            (decision_number, number_of_decisions - decision_number),
            mode="constant",
            constant_values=np.nan,
        )

        # Append to decision list
        self.decision_list.append(optimal_choice_padded)


class LifeSimulation:
    def __init__(
        self,
        people: list[Person],
        alpha: float,
        n_choices: int,
    ):
        self.people = people
        self.n_choices = n_choices

    def add_person(self, person: Person):
        self.people.append(person)

    def get_people(self) -> list[Person]:
        return self.people

    def get_person_by_name(self, name: str) -> Person | None:
        for person in self.people:
            if person.name == name:
                return person
        return None

    def get_number_of_people(self) -> int:
        return len(self.people)

    def generate_random_choice_function(seed=None):
        x = sp.symbols("x")

        if seed:
            np.random.seed(seed)
        m1 = np.random.uniform(0, 10)
        m2 = np.random.uniform(10, 20)
        s1 = np.random.uniform(1, 5)
        s2 = np.random.uniform(1, 5)
        a1 = np.random.uniform(-10, 10)
        a2 = np.random.uniform(-10, 10)

        y = a1 * sp.exp(-((x - m1) ** 2) / (2 * s1**2)) + a2 * sp.exp(
            -((x - m2) ** 2) / (2 * s2**2)
        )

        return {"m1": m1, "m2": m2, "s1": s1, "s2": s2, "a1": a1, "a2": a2, "expr": y}

    def generate_choice_array(self, n_choices: int):
        x = sp.symbols("x")

        # Generate n random functions (seeds for reproducibility)
        functions = [
            self.generate_random_choice_function(seed=i) for i in range(n_choices)
        ]

        # Create x values (100 points from 0 to 100)
        x_vals = np.linspace(0, 100, 100)

        # Build NumPy array: n rows (functions) × 100 columns (y values)
        choice_array = np.zeros((n_choices, 100))
        for i, func in enumerate(functions):
            y_func = sp.lambdify(x, func["expr"], "numpy")
            choice_array[i, :] = y_func(x_vals)
        return choice_array

    def simlulate_life(self, number_of_decisions: int):
        life_score_history = {"decision_number": list[int](range(number_of_decisions))}
        for person in self.people:
            person.initialise_credit_bank(self.n_credits, self.delay)
            life_score_history[person.name] = []
            for decision_number in range(number_of_decisions):
                choice_array = self.generate_choice_array(
                    self.n_choices, self.n_credits
                )
                person.calc_new_life_score(decision_number, choice_array)
                life_score_history[person.name].append(person.current_life_score)
        return pd.DataFrame(life_score_history)


def main():
    # Define people
    people = [
        Person("Amy", 1),
        Person("Barry", 2),
        Person("Cindy", 3),
        Person("Danny", 4),
    ]

    # Define simulation configuration
    n_choices = 10
    number_of_decisions = 500

    # Run simulation
    simulation = LifeSimulation(people, n_choices)
    life_score_history_df = simulation.simlulate_life(number_of_decisions)
    print(life_score_history_df)

    fig = px.line(
        life_score_history_df,
        x="decision_number",
        y=life_score_history_df.columns,
        title="Life Score History",
    )
    fig.show()


if __name__ == "__main__":
    main()

TypeError: LifeSimulation.__init__() missing 1 required positional argument: 'n_choices'